In [1]:
import geopandas as gpd
import os

# --- 1. 设置路径 ---
BOUNDARY_PATH = 'inputs/boundaries/university_city_boundary.geojson'
PARQUET_PATH = 'inputs/download/w080_n40_w075_n35.parquet'
OUTPUT_PATH = 'inputs/processed/University_City_Buildings_Clipped.geojson'

# 确保输出文件夹存在
os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

# --- 2. 读取边界 ---
print(f"正在读取边界: {BOUNDARY_PATH} ...")
boundary = gpd.read_file(BOUNDARY_PATH)

# 确保边界有坐标系 (默认为 EPSG:4326)
if boundary.crs is None:
    boundary.set_crs(epsg=4326, inplace=True)

# --- 3. 读取完整的 Parquet 文件 (修改点) ---
print(f"正在读取完整的建筑物文件 (这可能需要 1-2 分钟，请耐心等待)...")
# ❌ 去掉了 bbox 参数，直接读取
buildings = gpd.read_parquet(PARQUET_PATH)

print(f"原始文件共包含: {len(buildings)} 栋建筑")

# --- 4. 坐标系对齐 ---
# 只有坐标系一致才能裁剪
if buildings.crs != boundary.crs:
    print(f"转换坐标系: 从 {buildings.crs} 到 {boundary.crs}")
    buildings = buildings.to_crs(boundary.crs)

# --- 5. 精确裁剪 (Clip) ---
print("正在使用边界进行裁剪...")
# 这一步会切掉所有不在红框里的建筑
buildings_clipped = gpd.clip(buildings, boundary)

print(f"✅ 裁剪完成！最终保留建筑数: {len(buildings_clipped)}")

# --- 6. 释放内存 (可选) ---
# 删除那个大文件变量，释放内存
del buildings 

# --- 7. 保存结果 ---
print(f"正在保存结果到: {OUTPUT_PATH}")
buildings_clipped.to_file(OUTPUT_PATH, driver='GeoJSON')
print("🎉 所有操作完成！")

正在读取边界: inputs/boundaries/university_city_boundary.geojson ...
正在读取完整的建筑物文件 (这可能需要 1-2 分钟，请耐心等待)...
原始文件共包含: 10435471 栋建筑
转换坐标系: 从 OGC:CRS84 到 EPSG:4326
正在使用边界进行裁剪...
✅ 裁剪完成！最终保留建筑数: 7717
正在保存结果到: inputs/processed/University_City_Buildings_Clipped.geojson
🎉 所有操作完成！


In [2]:
!pip install rasterstats

In [3]:
import geopandas as gpd
import rasterio
from rasterstats import zonal_stats
import pandas as pd
import os

# --- 1. 设置文件路径 ---
# 使用之前裁剪好的小文件
vector_path = 'inputs/processed/University_City_Buildings_Clipped.geojson'
raster_path = 'inputs/download/University_City_Height.tif'
output_path = 'inputs/processed/University_City_Buildings_with_Height.geojson'

# --- 2. 读取数据 ---
print("正在读取建筑物数据...")
gdf = gpd.read_file(vector_path)

print("正在读取高度图信息...")
with rasterio.open(raster_path) as src:
    raster_crs = src.crs
    print(f"高度图坐标系为: {raster_crs}")

# --- 3. 关键修复：坐标系对齐 ---
# 如果建筑物的坐标系和高度图不一样，强制把建筑物“搬”到高度图的坐标系里
if gdf.crs != raster_crs:
    print(f"⚠️ 坐标系不一致 (矢量: {gdf.crs} vs 栅格: {raster_crs})")
    print("正在进行投影转换 (Reprojecting)...")
    gdf = gdf.to_crs(raster_crs)
    print("✅ 转换完成！现在两者重叠了。")
else:
    print("坐标系一致，无需转换。")

# --- 4. 核心计算：提取高度 ---
print("正在提取每栋建筑的高度 (Zonal Statistics)...")
# 我们只提取 'mean' (平均高度) 和 'max' (最大高度)
stats = zonal_stats(gdf, raster_path, stats=['mean', 'max'])
stats_df = pd.DataFrame(stats)

# --- 5. 赋值与清洗 ---
print("正在整理数据...")
gdf['height_m'] = stats_df['mean']

# 修复数据类型：将非数字的杂质转换为 NaN，然后填补为 0
gdf['height_m'] = pd.to_numeric(gdf['height_m'], errors='coerce').fillna(0)

# 剔除高度为 0 的建筑 (通常是极小的碎片或位于数据边缘)
before_count = len(gdf)
gdf = gdf[gdf['height_m'] > 0].copy() # 使用 .copy() 防止警告
print(f"清洗掉了 {before_count - len(gdf)} 个无高度数据的图斑，剩余 {len(gdf)} 栋建筑。")

# --- 6. 计算楼层数 ---
# 假设每层 3 米
gdf['floors_est'] = (gdf['height_m'] / 3.0).round(1)

# --- 7. (可选) 转回经纬度保存 ---
# 为了方便在某些 Web 地图或 GeoJSON 查看器中使用，通常最后转回 EPSG:4326
# 如果您后续还在 QGIS 用，这步不执行也可以，但为了通用性建议执行
if gdf.crs != "EPSG:4326":
    print("正在将结果转回经纬度坐标 (EPSG:4326) 以便保存...")
    gdf = gdf.to_crs("EPSG:4326")

# --- 8. 保存最终成品 ---
gdf.to_file(output_path, driver='GeoJSON')

print("------------------------------------------------")
print("🎉 大功告成！")
print(f"最终文件已保存: {output_path}")
print("您可以把这个文件拖进 QGIS，使用 '2.5D' 视图查看 3D 效果了！")
print("前 5 行数据预览：")
print(gdf[['height_m', 'floors_est', 'geometry']].head())

正在读取建筑物数据...
正在读取高度图信息...
高度图坐标系为: EPSG:32618
⚠️ 坐标系不一致 (矢量: EPSG:4326 vs 栅格: EPSG:32618)
正在进行投影转换 (Reprojecting)...
✅ 转换完成！现在两者重叠了。
正在提取每栋建筑的高度 (Zonal Statistics)...


C:\Users\Asus\anaconda3\envs\geoai\Lib\site-packages\rasterstats\io.py:335: NodataWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn(


正在整理数据...
清洗掉了 8 个无高度数据的图斑，剩余 7709 栋建筑。
正在将结果转回经纬度坐标 (EPSG:4326) 以便保存...
------------------------------------------------
🎉 大功告成！
最终文件已保存: inputs/processed/University_City_Buildings_with_Height.geojson
您可以把这个文件拖进 QGIS，使用 '2.5D' 视图查看 3D 效果了！
前 5 行数据预览：
   height_m  floors_est                                           geometry
0  7.314592         2.4  POLYGON ((-75.21523 39.94353, -75.21515 39.943...
1  6.726797         2.2  POLYGON ((-75.21504 39.94368, -75.21488 39.943...
2  6.128227         2.0  POLYGON ((-75.21549 39.94358, -75.21553 39.943...
3  7.537567         2.5  POLYGON ((-75.21502 39.94369, -75.21494 39.943...
4  6.645070         2.2  POLYGON ((-75.2155 39.94372, -75.21537 39.9438...


In [4]:
import os
import rasterio
import geopandas as gpd
from rasterio.mask import mask
from shapely.geometry import box

# ==========================================
# 1. 配置路径 (根据你现在的实际情况)
# ==========================================

# 边界文件
BOUNDARY_PATH = 'inputs/boundaries/university_city_boundary.geojson'

# 原始航片 (很大)
AERIAL_SRC    = 'inputs/aerial/2017_4BandImagery_J1288816/2017_4BandImagery_J1288816tR0_C0.tif' 

# 下载并处理好的高度图 (现成的 DSM)
DSM_PATH      = 'inputs/download/University_City_Height.tif'

# 处理好的建筑物矢量 (成品)
BUILDING_PATH = 'inputs/processed/University_City_Buildings_with_Height.geojson'

# 输出路径
OUTPUT_DIR    = 'outputs/processed_rasters'
os.makedirs(OUTPUT_DIR, exist_ok=True)
AERIAL_FINAL  = os.path.join(OUTPUT_DIR, 'study_area_Aerial.tif')

# ==========================================
# 2. 裁剪函数 (保留这个工具)
# ==========================================
def crop_raster(input_raster, boundary_gdf, output_path):
    print(f"📷 正在裁剪航片: {os.path.basename(input_raster)}")
    
    with rasterio.open(input_raster) as src:
        # 统一坐标系
        if boundary_gdf.crs != src.crs:
            print(f"   转换边界坐标系到 {src.crs}...")
            boundary_gdf = boundary_gdf.to_crs(src.crs)
            
        try:
            # 执行裁剪
            out_image, out_transform = mask(src, boundary_gdf.geometry, crop=True)
            out_meta = src.meta.copy()
            out_meta.update({
                "driver": "GTiff",
                "height": out_image.shape[1],
                "width": out_image.shape[2],
                "transform": out_transform
            })
            
            with rasterio.open(output_path, "w", **out_meta) as dest:
                dest.write(out_image)
            print(f"✅ 航片裁剪成功: {output_path}")
            
        except ValueError:
            print("❌ 裁剪失败: 航片与边界没有重叠区域。")

# ==========================================
# 3. 主流程
# ==========================================

print("🚀 项目数据检查与准备...")

# 1. 检查核心矢量数据
if os.path.exists(BUILDING_PATH):
    gdf = gpd.read_file(BUILDING_PATH)
    print(f"✅ 建筑物数据就绪: {len(gdf)} 栋 (已包含高度属性)")
else:
    print(f"❌ 警告: 找不到建筑物文件 {BUILDING_PATH}，请先运行上一步的高度提取代码！")

# 2. 检查现成的高度图
if os.path.exists(DSM_PATH):
    print(f"✅ 高度图数据就绪: {os.path.basename(DSM_PATH)}")
else:
    print(f"❌ 警告: 找不到高度图 {DSM_PATH}")

# 3. 处理航片 (只有这个需要重新跑)
if not os.path.exists(AERIAL_FINAL):
    if os.path.exists(AERIAL_SRC):
        boundary = gpd.read_file(BOUNDARY_PATH)
        crop_raster(AERIAL_SRC, boundary, AERIAL_FINAL)
    else:
        print(f"❌ 找不到原始航片文件，跳过裁剪。")
else:
    print(f"✅ 裁剪后的航片已存在: {AERIAL_FINAL}")

print("\n🎉 所有数据准备完毕！你可以开始做可视化分析了。")

🚀 项目数据检查与准备...
✅ 建筑物数据就绪: 7709 栋 (已包含高度属性)
✅ 高度图数据就绪: University_City_Height.tif
✅ 裁剪后的航片已存在: outputs/processed_rasters\study_area_Aerial.tif

🎉 所有数据准备完毕！你可以开始做可视化分析了。
